<a href="https://colab.research.google.com/github/feaviolp/msc-project/blob/main/NIJ%20EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exploratory Data Analysis on AB_NYC_2019 dataset

## This notebook consists of several sections:

### - Data preprocessing
### - Exploratory Data Analysis, including modifications, correlation analysis, and visualisations
### - Clustering:
    - K Prototype
    - K Means
    - DBSCAN
### - Linear Regression

## Import Libraries

In [16]:
%pip install kmodes

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
import scipy.stats as st
from sklearn import linear_model
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.metrics import r2_score
from sklearn.metrics import silhouette_score
from kmodes.kprototypes import KPrototypes
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import LabelEncoder, StandardScaler
import warnings
# ignore future deprecation
warnings.filterwarnings('ignore')

Read the AB_NYC_2019.csv file

In [ ]:
airbnb = pd.read_csv("AB_NYC_2019.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'AB_NYC_2019.csv'

## Data Preprocessing

Take a look at the dataset.

In [ ]:
airbnb.info()

The dataset has 48,895 rows and 16 columns.

We can see that some of the columns have missing values since they have less than 48,895 entries.

Describe the dataset

In [ ]:
airbnb.describe()

Look at the top and bottom 5 rows of data and look at the shape of the data frame (number of rows and columns)

In [ ]:
airbnb

In [ ]:
airbnb.shape

checks for duplicates in dataframe

In [ ]:
airbnb.duplicated().sum()

Show the number of missing values per variable

In [ ]:
airbnb.isna().sum()

There are a lot of missing variables, especially last_review and reviews_per_month.

Replace null values with appropriate values:
- name is categorical so will simply be replaced with "Replaced name"
- host_name is categorical so will simply be replaced with "Replaced host name"
- last_review is date so will be replaced with 0 (0 is not ideal for dates but we won't be using this variable, null value is replaced to avoid any errors)
- review_per_month is a continuous variable so will be replaced with 0

In [ ]:
airbnb['name'].fillna('Replaced name', inplace=True)
airbnb['host_name'].fillna('Replaced host name', inplace=True)
airbnb['last_review'].fillna(0, inplace=True)
airbnb['reviews_per_month'].fillna(0, inplace=True)

Now re-check to make sure there are no missing values

In [ ]:
airbnb.isna().sum()

In [ ]:
airbnb.id.duplicated().sum()

Check the 5-figure summary for price and availability_365

In [ ]:
airbnb.price.describe()

In [ ]:
airbnb.availability_365.describe()

Availability_365 is the number of days in the next year that the room is available to be booked, including days already booked, so a figure of zero means the room is currently not available.

A 0 for price also suggests something not right with that listing.

To tidy the data we therefore drop all rows with availability_365=0 or price=0

In [ ]:
airbnb.drop(airbnb[airbnb.price == 0].index, inplace=True)
airbnb.drop(airbnb[airbnb.availability_365 == 0].index, inplace=True)

In [ ]:
airbnb.shape

In [ ]:
airbnb.price.describe()

In [ ]:
airbnb.availability_365.describe()

We can see that 17,541 rows have been dropped and we no longer have any zero entries for price or availability_365.

## Exploratory Data Analysis

Examine numerical features in the dataset

In [ ]:
numeric_features = airbnb.select_dtypes(include=[np.number])
numeric_features.columns

Examine categorical features in the dataset

In [ ]:
categorical_features = airbnb.select_dtypes(include=[object])
categorical_features.columns

Counting rows per categorical variable

In [ ]:
airbnb.host_name.value_counts()

In [ ]:
airbnb.name.value_counts()

In [ ]:
airbnb.neighbourhood_group.value_counts()

In [ ]:
airbnb.neighbourhood.value_counts()

In [ ]:
airbnb.room_type.value_counts()

In [ ]:
airbnb.last_review.value_counts()

Estimate Skewness and Kurtosis

In [ ]:
airbnb.skew()

In [ ]:
airbnb.kurt()

Now check to see if price is normaly distributed

In [ ]:
y = airbnb['price']
plt.figure(1); plt.title('Johnson SU')
sns.distplot(y, kde=False, fit=st.johnsonsu)
plt.figure(2); plt.title('Normal')
sns.distplot(y, kde=False, fit=st.norm)
plt.figure(3); plt.title('Log Normal')
sns.distplot(y, kde=False, fit=st.lognorm)

We can see that price is heavily skewed so we'll find the five-figure summary for price.

In [ ]:
airbnb.price.describe()

The five figure summary shows the range is 10-10000 with Q3 being 189. So 75 percent of the properties fall between 10 and 189. Check the distribution of price for only properties equal to or below 189 (75 percent of the sample)

In [ ]:
low_price = airbnb[airbnb['price'] <= 189]
y = low_price['price']
plt.figure(1); plt.title('Johnson SU')
sns.distplot(y, kde=False, fit=st.johnsonsu)
plt.figure(2); plt.title('Normal')
sns.distplot(y, kde=False, fit=st.norm)
plt.figure(3); plt.title('Log Normal')
sns.distplot(y, kde=False, fit=st.lognorm)

There is still a lot of variation.

Let's try to make it better with a log transformation

In [ ]:
airbnb['log_price'] = np.log(airbnb['price'])

Visualise transformed log distribution

In [ ]:
plt.hist(airbnb['log_price'], bins=30)
plt.xlabel('Log Price')
plt.ylabel('Frequency')
plt.title('Distribution of Log Price')
plt.show()

That's better - we can work with that.

Price is the charge per night which isn't a particularly helpful figure for Airbnb on its own.

We will therefore calculate the total revenue opportunity to Airbnb for each room if it were to be booked for every day that it is available. This is of course unlikely, but it is a useful comparrison as to the potential maximum revenue of each room. The commission for each room is 17% of the price charged, made-up of 3% host fee and 14% guest fee: https://www.airbnb.co.uk/resources/hosting-homes/a/how-much-does-airbnb-charge-hosts-288

Revenue_opportunity = price * availability_365 * 0.17

In [ ]:
airbnb['revenue_opportunity'] = airbnb['price'] * airbnb['availability_365'] * 0.17

In [ ]:
airbnb.head()

Now see if revenue_opportunity is normaly distributed

In [ ]:
y = airbnb['revenue_opportunity']
plt.figure(1); plt.title('Johnson SU')
sns.distplot(y, kde=False, fit=st.johnsonsu)
plt.figure(2); plt.title('Normal')
sns.distplot(y, kde=False, fit=st.norm)
plt.figure(3); plt.title('Log Normal')
sns.distplot(y, kde=False, fit=st.lognorm)

It isn't, so let's log transform as we did with price.

In [ ]:
airbnb['log_revenue_opportunity'] = np.log(airbnb['revenue_opportunity'])

In [ ]:
plt.hist(airbnb['log_revenue_opportunity'], bins=30)
plt.xlabel('Log Revenue Opportunity')
plt.ylabel('Frequency')
plt.title('Distribution of Log Revenue Opportunity')
plt.show()

That's better. Now we'll add the new log variables to the numeric features

In [ ]:
numeric_features = airbnb.select_dtypes(include=[np.number])
numeric_features.columns

## Correlations

Correlation matrix with price

In [ ]:
correlation = numeric_features.corr()
correlation['price'].sort_values(ascending = False)

Correlation matrix with log price

In [ ]:
correlation = numeric_features.corr()
correlation['log_price'].sort_values(ascending = False)

To explore further we will start with the following visualisation methods to analyze the data better:

 - Correlation Heat Map
 - Zoomed Heat Map
 - Pair Plot
 - Scatter Plot

In [ ]:
sns.pairplot(airbnb)

### Correlation Heat Map

In [ ]:
f , ax = plt.subplots(figsize = (14,12))
plt.title('Correlation of Numeric Features',y=1,size=16)
sns.heatmap(correlation,square = True)

The only strong correlations appear to be number_of_reviews with id, which is not likely to be helpful, and number_of_review with reviews_per_month.


### Zoomed HeatMap

In [ ]:
k= 11
cols = correlation.nlargest(k,'price')['price'].index
print(cols)
cm = np.corrcoef(airbnb[cols].values.T)
f , ax = plt.subplots(figsize = (14,12))
sns.heatmap(cm, vmax=.8, linewidths=0.01,square=True,annot=True,cmap='viridis',
            linecolor="white",xticklabels = cols.values ,annot_kws = {'size':12},yticklabels = cols.values)

Price isn't highly correlated with any variables other than the ones we added which were derrived from price anyway.

So regression is unlikely to be much help here.

Bar chart of median price by neighbourhood group

In [ ]:
neighbourhood_group = airbnb.pivot_table(index ='neighbourhood_group',values = 'price', aggfunc = np.median)
neighbourhood_group.plot(kind = 'bar',color = 'blue')
plt.xlabel('Neighbourhood group')
plt.ylabel('Median price')

We can see that Manhattan has the highest median price by some distance.

Bar chart of mean price by neighbourhood group

In [ ]:
neighbourhood_group = airbnb.pivot_table(index ='neighbourhood_group',values = 'price', aggfunc = np.mean)
neighbourhood_group.plot(kind = 'bar',color = 'blue')
plt.xlabel('Neighbourhood group')
plt.ylabel('Mean price')

Manhattan also has the highest mean price

Now lets see the revenue opportunity

In [ ]:
neighbourhood_group = airbnb.pivot_table(index ='neighbourhood_group',values = 'revenue_opportunity', aggfunc = np.sum)
neighbourhood_group.plot(kind = 'bar')
plt.xlabel('Neighbourhood group')
plt.ylabel('Revenue opportunity @ 17% commission')

In [ ]:
neighbourhood_group = airbnb.pivot_table(index ='neighbourhood_group',values = 'id', aggfunc = 'count')
neighbourhood_group.plot(kind = 'bar',color = 'blue')
plt.xlabel('Neighbourhood group')
plt.ylabel('Number of properties')

Interesting that Brooklyn has half the revenue opportunity of Manhattan eventhough it has almost as many properties.

Let's take a look at availability_365 to see what's happening.

In [ ]:
var = 'neighbourhood_group'
data = pd.concat([airbnb['availability_365'], airbnb[var]], axis=1)
f, ax = plt.subplots(figsize=(12, 8))
fig = sns.boxplot(x=var, y="availability_365", data=data)

We can see the reason now. Brooklyn has the lowest availability of all groups.

In [ ]:
airbnb.groupby('neighbourhood_group')['availability_365'].describe()

Now let's see the total number of rooms and average revenue opportunity per property by neighbourhood group

Lets calculate the number of rooms per person using 2020 data (source: https://www.census.gov/programs-surveys/decennial-census/decade/2020/2020-census-results.html) that says populations are:
- Manhattan = 1.629m
- Brooklyn = 2.577m
- Queens = 2.271m
- Bronx = 1.476m
- Staten Island = 476k

In [ ]:
neighbourhoods = dict()

for item in set(airbnb.neighbourhood_group):
    neighbourhoods[item]=(airbnb[airbnb.neighbourhood_group == item].shape[0])

for key in neighbourhoods:
    match key:
        case 'Staten Island':
            neighbourhoods[key] = round(neighbourhoods[key]/476000,5)
        case 'Manhattan':
            neighbourhoods[key] = round(neighbourhoods[key]/1629000,5)
        case 'Queens':
            neighbourhoods[key] = round(neighbourhoods[key]/2271000,5)
        case 'Brooklyn':
            neighbourhoods[key] = round(neighbourhoods[key]/2577000,5)
        case 'Bronx':
            neighbourhoods[key] = round(neighbourhoods[key]/1476000,5)

for key in neighbourhoods:
    print('In {}, based on the population and the listings we have a corresponding {} listings per person'.format(key, neighbourhoods[key]))

So Manhatten has the highest utilisation. Does that mean there is the lowest chance of increasing penetration there? What would it look like if every other neighbourhood group could reach the same 0.00832 rooms per person at their current mean revenue opportunity?

In [ ]:
airbnb.groupby('neighbourhood_group')['revenue_opportunity'].describe()

Total people living in each group * 0.00832:

In [ ]:
# Manhattan
print("Manhattan: already already at 0.00832 listings per person (13,559 listings)")

# Brooklyn
print("Brooklyn: ", round(2577000 * 0.00832))

# Queens
print("Queens: ", round(2271000 * 0.00832))

# Bronx
print("Bronx: ", round(1476000 * 0.00832))

# Staten Island
print("Staten Island: ", round(476000 * 0.00832))

So... if they all achieved those rates it would look like this (rooms * average revenue):

In [ ]:
# Manhattan
print("Manhattan: ", round(13559 * 7226.5))

# Brooklyn
print("Brooklyn: ", round(21441 * 3882.1))

# Queens
print("Queens: ", round(18895 * 3420.9))

# Bronx
print("Bronx: ", round(12280 * 3145.9))

# Staten Island
print("Staten Island: ", round(3960 * 4656.8))

Subtracting actual from potential. First check actual potential revenue:

In [ ]:
airbnb.pivot_table(index ='neighbourhood_group',values = 'revenue_opportunity', aggfunc = np.sum)


- Manhatten = USD 97.9m - USD 97.9m = USD 0
- Brooklyn = USD 83.2m - USD 47.6mm = USD 35.6m
- Queens = USD 64.6m - USD 14.7m = USD 49.9m
- Bronx = USD 38.6m - USD 2.9m = USD 35.7m
- Staten Island = USD 18.4m - USD 1.5m = USD 16.9m

Based on this, Queens has the highest potential additional revenue even though it has the second lowest mean revenue opportunity because it has the lowest number of rooms available per resident, and so the biggest opportunity to expand.

# Tables for the report - different stylings

### table 1

In [ ]:
# Create new dataframe, .reset_index() to convert series into dataframe
counts_df = airbnb.neighbourhood_group.value_counts().reset_index()
counts_df.columns = ['Neighbourhood Group', 'Listings']

# Create population dict from cencus data
population_dict = {
    'Manhattan': 1.629e6,
    'Brooklyn': 2.577e6,
    'Queens': 2.271e6,
    'Bronx': 1.476e6,
    'Staten Island': 476e3
}

# Add population column to df
counts_df['Population'] = counts_df['Neighbourhood Group'].map(population_dict).apply(lambda x: int(x))

# Add listings per resident to df
counts_df['Listings per Resident'] = (counts_df['Listings'] / counts_df['Population']).round(5)

counts_df

In [ ]:
styled_df = counts_df.style.set_properties(**{'background-color': '#F0FFFF',
                                              'color': 'black',
                                              'border-color': 'white'}).format({"Listings": "{:,.0f}",
                                                                                "Population": "{:,.0f}",
                                                                                "Listings per Resident": "{:.5g}"})

styled_df.hide_index()

In [ ]:
styled_df = counts_df.style.background_gradient(
    cmap='Blues'
).hide_index().format(
    {
        "Listings": "{:,.0f}",
        "Population": "{:,.0f}",
        "Listings per Resident": "{:.5g}"
    }
).set_table_styles(
    [
        {
            'selector': 'th',
            'props': [
                ('padding', '1px'),
                ('text-align', 'left')
            ]
        },
        {
            'selector': 'td',
            'props': [
                ('padding', '1px'),
                ('text-align', 'center')
            ]
        }
    ]
).set_properties(**{'width': '70px'})

styled_df

### table 2

create rev opp dict

In [ ]:
mean_revenue_dict = airbnb.groupby('neighbourhood_group')['revenue_opportunity'].mean().to_dict()
mean_revenue_dict

revenue_opportunity_dict = airbnb.pivot_table(index='neighbourhood_group', values='revenue_opportunity', aggfunc=np.sum).to_dict()['revenue_opportunity']
revenue_opportunity_dict

In [ ]:
table_2 = pd.DataFrame()

# Build columns
table_2['Neighbourhood Group'] = counts_df['Neighbourhood Group']
table_2['Listings * 0.00832'] = counts_df['Population'] * 0.00832
table_2['Mean Revenue'] = table_2['Neighbourhood Group'].map(mean_revenue_dict)
table_2['Current Revenue Opportunity'] = table_2['Neighbourhood Group'].map(revenue_opportunity_dict)
table_2['Potential Revenue * 0.00832'] = table_2['Mean Revenue'] * table_2['Listings * 0.00832']

# Hardcode Manhattan back since that is the baseline and rounding changes the value
table_2.loc[table_2['Neighbourhood Group'] == 'Manhattan', 'Potential Revenue * 0.00832'] = 97984186.9

table_2['Potential Revenue Increase'] = table_2['Potential Revenue * 0.00832'] - table_2['Current Revenue Opportunity']

table_2

In [ ]:
# Currency format func
def format_currency_in_millions(value):
    return "$ {:.1f}M".format(value / 1_000_000)

# format
styled_table_2 = table_2.style.format({
    'Listings * 0.00832': "{:,.0f}",
    'Mean Revenue': "$ {:,.0f}",
    'Current Revenue Opportunity': format_currency_in_millions,
    'Potential Revenue * 0.00832': format_currency_in_millions,
    'Potential Revenue Increase': format_currency_in_millions
}).background_gradient(cmap='Blues', subset=['Listings * 0.00832', 'Mean Revenue', 'Current Revenue Opportunity', 'Potential Revenue * 0.00832', 'Potential Revenue Increase']).hide_index().set_table_styles([
   {
      'selector': 'th',
      'props': [
          ('padding', '1px'),
          ('text-align', 'left')
      ]
   },
   {
      'selector': 'td',
      'props': [('padding', '1px'),
        ('text-align', 'center')]
   },
]).set_properties(**{'width': '60px'})

styled_table_2


Style and reformat table

In [ ]:
styled_table_2 = table_2.style.format({
    'Listings * 0.00832': "{:,.0f}",
    'Mean Revenue': "$ {:,.0f}",
    'Current Revenue Opportunity': "$ {:,.0f}",
    'Potential Revenue * 0.00832': "$ {:,.0f}",
    'Potential Revenue Increase': "$ {:,.0f}"
}).background_gradient(cmap='Blues').hide_index().set_table_styles([
   {
      'selector': 'th',
      'props': [
          ('padding', '1px'),
          ('text-align', 'left')
      ]
   },
   {
      'selector': 'td',
      'props': [('padding', '1px')]
   },
]).set_properties(**{'width': '100px'})

styled_table_2

Box plot price by neighbourhood group

In [ ]:
var = 'neighbourhood_group'
data = pd.concat([airbnb['price'], airbnb[var]], axis=1)
f, ax = plt.subplots(figsize=(12, 8))
fig = sns.boxplot(x=var, y="price", data=data)

The extremes are so high the chart cannot be read easily, so re-plot with maximum price of 500

In [ ]:
var = 'neighbourhood_group'
data = pd.concat([airbnb['price'], airbnb[var]], axis=1)
f, ax = plt.subplots(figsize=(12, 8))
fig = sns.boxplot(x=var, y="price", data=data)
fig.axis(ymin=0, ymax=500);

We can see again that Manhattan has the highest prices, showing that all figures in the five figure summary are higher than any other neighrohood group.

Bar chart of number of properties per neighbourhood group

In [ ]:
neighbourhood_group = airbnb.pivot_table(index ='neighbourhood_group',values = 'id', aggfunc = 'count')
neighbourhood_group.plot(kind = 'bar',color = 'blue')
plt.xlabel('Neighbourhood group')
plt.ylabel('Number of properties')

We can see that Manhattan has the most roperties, closely follwed by Brooklyn. So Manhattan has the most properties, and they are the most expensive.

Break-down Manhattan with a box plot of price by neighbourhood in Manhattan with a max of 1000 for price for readability

In [ ]:
Manhattan = airbnb[airbnb['neighbourhood_group'] == 'Manhattan']
var = 'neighbourhood'
data = pd.concat([Manhattan['price'], Manhattan[var]], axis=1)
f, ax = plt.subplots(figsize=(12, 8))
fig = sns.boxplot(x=var, y="price", data=data)
fig.axis(ymin=0, ymax=1000);
plt.xticks(rotation=90)

We can see that within Manhattan there is a lot of price variation between the neighbourhoods.

Bar chart of number of properties per neighbourhood in Manhattan

In [ ]:
neighbourhood_group = Manhattan.pivot_table(index ='neighbourhood',values = 'id', aggfunc = 'count')
neighbourhood_group.plot(kind = 'bar',color = 'blue')
plt.xlabel('Neighbourhood')
plt.ylabel('Number of properties')

We can also see a large variation between the number of properties listed in each neighbourhood.

At first glance it apears that neighbourhoods with the higher prices have fewer properties, but more detailed analysis would be needed in this areas.

## Exploring the variable room_type:

Count the number of each room type

In [ ]:
airbnb.room_type.value_counts()

Now show them by neightborhood group

In [ ]:
# plot barchart with neighbourhood_group and room_type counts
grouped = airbnb.groupby(['neighbourhood_group', 'room_type']).size().reset_index(name='counts')

# Create a figure and a set of subplots
fig, ax = plt.subplots(figsize=(10, 5))

# Create the bar chart
sns.barplot(x='neighbourhood_group', y='counts', hue='room_type', data=grouped, ax=ax)

# Rotate x-axis labels for better visibility
plt.xticks(rotation=90)

# Display the plot
plt.show()

Interestingly Queens has a lower proportion of private rooms to entire home/appt than Mahattan or Brooklyn. Increasing the number of entire home/appt in Queens could be another potential area for Airbnb to explore.

Boxplot for room_type vs price

In [ ]:
sns.boxplot(x='room_type', y='price', data=airbnb)
plt.title('Room Type vs Price')
plt.xticks(rotation=90)
plt.axis(ymin=0, ymax=500);
plt.show()

 Grouped bar chart showing counts of room types in different neighbourhood groups

In [ ]:
room_counts = airbnb.groupby(['neighbourhood_group', 'room_type']).size().unstack()
room_counts.plot(kind='bar', stacked=True)
plt.title('Counts of Room Types in Different Neighbourhood Groups')
plt.show()

The analysis shows:
There are more entire home/apt, followed by private rooms, followed by shared room.
This is also true in Manhattan and Brooklyn, where there are most properties available, but not in Queens where there are more private rooms than entire suite/apts.
The entire suite/apt costs most, followed by private room, followed by shared room.

## Now plot some results on the map

Plot the properties on the map, grouped by neighbourhood_group, to visualise the distribution.

In [ ]:
plt.figure(figsize=(12,8))
plt.style.use('fast')
# Set the boundary of the map using longitude and latitude obtained from Google Maps
coordinates = (-74.2623, -73.6862, 40.4943, 40.9144)
map = mpimg.imread("New_York_City.jpg")
plt.imshow(map,extent=coordinates)
groups = airbnb.groupby('neighbourhood_group')
for name,group in groups :
     plt.scatter(group['longitude'],group['latitude'], label=name, edgecolors='black')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Properties by neighbourhood')
plt.legend()

Now plot the properties using a colour gradiant for price

In [ ]:
plt.figure(figsize=(12,8))
plt.imshow(map, extent=coordinates)
plt.scatter(airbnb.longitude, airbnb.latitude, c=airbnb.price, cmap='rainbow', edgecolors='black')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Properties by price')
plt.colorbar()

The wide price range makes this view unhelpful. We know the range is 0-10,000 but 75% of the properties are below 189, so the vast majority of the properties will have tiny colour variations at the same end of the colour range.

We'll take a look at the higher value properties first; plot only those over 1000

In [ ]:
plt.figure(figsize=(12,8))
plt.imshow(map, extent=coordinates)
high_price = airbnb[airbnb['price'] > 1000]
plt.scatter(high_price.longitude, high_price.latitude, c=high_price.price, cmap='rainbow', edgecolors='black')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Properties over $1000 by price')
plt.colorbar()

We can start to see where the very expensive properties are located, which is Manhatten as expected from the earlier analysis of price by neighbourhood_region, but also Queens which was unexpected.

Now to see the price distribution of the lower cost properties. We know that 75% of properties are priced 189 or below so plot only those properties.

In [ ]:
plt.figure(figsize=(12,8))
plt.imshow(map, extent=coordinates)
low_price = airbnb[airbnb['price'] <= 189]
plt.scatter(low_price.longitude, low_price.latitude, c=low_price.price, cmap='rainbow', edgecolors='black')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Properties under $189 by price')
plt.colorbar()

Once again the higher priced properties at the lower end of the price range are clustered in Manhatten.

Now to see the price distribution of the middle 50% of properties by cost, plot only those >= 70 and <= 189

In [ ]:
plt.figure(figsize=(12,8))
plt.imshow(map, extent=coordinates)
mid_price = airbnb[airbnb['price'] >= 70]
mid_price = mid_price[mid_price['price'] <= 189]
plt.scatter(mid_price.longitude, mid_price.latitude, c=mid_price.price, cmap='rainbow', edgecolors='black')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Properties >= $70 and <= $189 by price')
plt.colorbar()

In the mid-price range the higher prices are again clusted in Manhatten, but they are distributed across the other regions too.

# Clustering

Using the log price for clustering to handle skewness, scale down values, and better interpretability for more meaningful clusters

## Starting with k-prototype `log_price`  `neighbourhood` `latitude` `longitude`

In [ ]:
# keeping less than we're dropping so just picking those features
cluster_data = airbnb[['log_price', 'neighbourhood', 'latitude', 'longitude']]
cluster_data

In [ ]:
# confirm datatypes
cluster_data.dtypes

In [ ]:
# Create a copy of the data
cluster_data_prepared = cluster_data.copy()

# Encode categorical variables
le = LabelEncoder()
cluster_data_prepared['neighbourhood'] = le.fit_transform(cluster_data['neighbourhood'])

cluster_data_prepared

### k-prototype clustering
k-prototype combines K-Means (for numerical features) and K-Modes (for categorical features).  K-Modes works best with label-encoded categorical variables so we will label encode rather than one-hot. Saves computational power too.

Performing the first run with `log_price` `neighbourhood` `latitude` `longitude`

In [ ]:
# copy original cluster data to rename it for this no var transformed run
kproto_prep_1 = cluster_data_prepared.copy()

# standardise numerical variables
scaler = StandardScaler()
kproto_prep_1[['log_price']] = scaler.fit_transform(cluster_data[['log_price']])
kproto_prep_1[['latitude']] = scaler.fit_transform(cluster_data[['latitude']])
kproto_prep_1[['longitude']] = scaler.fit_transform(cluster_data[['longitude']])

# Keep the original latitude and longitude
orig_lat = cluster_data['latitude'].copy()
orig_long = cluster_data['longitude'].copy()

kproto_prep_1

In [ ]:
k_values = []
sil_scores = []
costs = []

for i in range(2,13):
    # Initialize K-Prototypes algorithm
    kproto = KPrototypes(n_clusters=i, init='Cao', verbose=0)

    # Fit and predict clusters
    # Specify categorical variables
    clusters = kproto.fit_predict(kproto_prep_1.values, categorical=[1])

    # Compute silhouette score
    SScore = silhouette_score(kproto_prep_1.values, clusters, metric='euclidean')

    k_values.append(i)
    sil_scores.append(SScore)
    costs.append(kproto.cost_)

    print("Silhouette score for k (clusters) = " + str(i) + " is " + str(SScore))

# Plot silhouette scores
plt.figure(figsize=(10,5))
plt.subplot(1, 2, 1)
plt.plot(k_values, sil_scores, 'bx-')
plt.xlabel('k (number of clusters)')
plt.ylabel('Silhouette Score')
plt.title('Silhouette Score vs Number of Clusters')

# Plot costs (for the elbow plot)
plt.subplot(1, 2, 2)
plt.plot(k_values, costs, 'bx-')
plt.xlabel('k (number of clusters)')
plt.ylabel('Cost')
plt.title('Elbow Plot (Cost vs Number of Clusters)')

plt.tight_layout()
plt.show()

## Above results show that 6 clusters are optimal, continue analysis:

Although not a great rating in general, we will continue with 6 for a fair compromise between silhouette score and elbow

In [ ]:
# Initialize K-Prototypes algorithm
kproto = KPrototypes(n_clusters=6, init='Cao', verbose=0)

# Fit and predict clusters
clusters = kproto.fit_predict(kproto_prep_1.values, categorical=[1])

# Add cluster assignments to original data
kproto_prep_1['cluster'] = clusters

In [ ]:
kproto_prep_1

measure cluster size

In [ ]:
kproto_prep_1['cluster'].value_counts()

## Plot map

In [ ]:
plt.figure(figsize=(12,8))
plt.style.use('fast')

# Set the boundary of the map using longitude and latitude obtained from Google Maps
coordinates = (-74.2623, -73.6862, 40.4943, 40.9144)

map = mpimg.imread("New_York_City.jpg")
plt.imshow(map,extent=coordinates)

# Group by cluster labels instead of neighbourhood group
clusters = kproto_prep_1.groupby('cluster')

# Loop through each cluster and plot the listings in it
for name, group in clusters:
    plt.scatter(group['longitude'], group['latitude'], label=name, edgecolors='black')

plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Properties by Cluster')
plt.legend()

Place original lat & long back so the results can be plotted on the map

In [ ]:
kproto_prep_1_org_lat_long = kproto_prep_1.copy()

kproto_prep_1_org_lat_long['latitude'] = orig_lat
kproto_prep_1_org_lat_long['longitude'] = orig_long

kproto_prep_1_org_lat_long

In [ ]:
plt.figure(figsize=(12,8))
plt.style.use('fast')

# Set the boundary of the map using longitude and latitude obtained from Google Maps
coordinates = (-74.2623, -73.6862, 40.4943, 40.9144)

map = mpimg.imread("New_York_City.jpg")
plt.imshow(map,extent=coordinates)

# Group by cluster labels instead of neighbourhood group
clusters = kproto_prep_1_org_lat_long.groupby('cluster')

# Loop through each cluster and plot the listings in it
for name, group in clusters:
    plt.scatter(group['longitude'], group['latitude'], label=name, edgecolors='black')

plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Properties by Cluster')
plt.legend()

This looks interesting. Lets explore the data with summary statistics for each cluster group

In [ ]:
cluster_summary = kproto_prep_1_org_lat_long.groupby('cluster')['log_price'].describe()
cluster_summary

In [ ]:
# Boxplot of 'log_price' for each cluster
plt.figure(figsize=(12, 8))
sns.boxplot(x='cluster', y='log_price', data=kproto_prep_1_org_lat_long)
plt.title('Price Distribution per Cluster')
plt.show()

## ANOVA results

In [ ]:
anova_results = st.f_oneway(*(kproto_prep_1_org_lat_long['log_price'][kproto_prep_1_org_lat_long['cluster'] == cluster] for cluster in kproto_prep_1_org_lat_long['cluster'].unique()))
anova_results

The above would suggest that the price is significantly different across the clusters.

In [ ]:
room_counts = kproto_prep_1_org_lat_long.groupby(['cluster', airbnb['room_type']]).size().unstack()
room_counts.plot(kind='bar', stacked=True)
plt.title('Counts of Room Types in Different Clusters')
plt.show()

## Add `neighbourhood_group` back to see which group is in which cluster

In [ ]:
# put neighbourhoods back
kproto_prep_1_org_lat_long['neighbourhood_group'] = airbnb['neighbourhood_group']

In [ ]:
kproto_prep_1_org_lat_long

In [ ]:
plt.figure(figsize=(10, 8))
sns.countplot(data=kproto_prep_1_org_lat_long, x='neighbourhood_group', hue='cluster')

plt.title('Distribution of Clusters within Each Neighbourhood Group')
plt.ylabel('Count')
plt.show()

# k-prototype `revenue_opportunity` `neighbourhood` `latitude` `longitude`

In [ ]:
kproto_prep_2 = airbnb[['log_revenue_opportunity', 'neighbourhood', 'latitude', 'longitude']]
kproto_prep_2

Standardise and label encode

In [ ]:
# Keep the original latitude and longitude
orig_lat = kproto_prep_2['latitude'].copy()
orig_long = kproto_prep_2['longitude'].copy()

# standardise numeric variables
scaler = StandardScaler()
kproto_prep_2[['latitude', 'longitude', 'log_revenue_opportunity']] = scaler.fit_transform(kproto_prep_2[['latitude', 'longitude', 'log_revenue_opportunity']])

# encode numerical variables
le = LabelEncoder()
kproto_prep_2['neighbourhood'] = le.fit_transform(kproto_prep_2['neighbourhood'])

In [ ]:
kproto_prep_2

In [ ]:
kproto_prep_2.dtypes

In [ ]:
k_values = []
sil_scores = []
costs = []

for i in range(2,13):
    # Initialize K-Prototypes algorithm
    kproto = KPrototypes(n_clusters=i, init='Cao', verbose=0)

    # Fit and predict clusters
    # Specify categorical variables
    clusters = kproto.fit_predict(kproto_prep_2.values, categorical=[1])

    # Compute silhouette score
    SScore = silhouette_score(kproto_prep_2.values, clusters, metric='euclidean')

    k_values.append(i)
    sil_scores.append(SScore)
    costs.append(kproto.cost_)

    print("Silhouette score for k (clusters) = " + str(i) + " is " + str(SScore))

# Plot silhouette scores
plt.figure(figsize=(10,5))
plt.subplot(1, 2, 1)
plt.plot(k_values, sil_scores, 'bx-')
plt.xlabel('k (number of clusters)')
plt.ylabel('Silhouette Score')
plt.title('Silhouette Score vs Number of Clusters')

# Plot costs (for the elbow plot)
plt.subplot(1, 2, 2)
plt.plot(k_values, costs, 'bx-')
plt.xlabel('k (number of clusters)')
plt.ylabel('Cost')
plt.title('Elbow Plot (Cost vs Number of Clusters)')

plt.tight_layout()
plt.show()

## Above results show that 6 clusters are optimal, continue analysis:

In [ ]:
# Initialize K-Prototypes algorithm
kproto = KPrototypes(n_clusters=6, init='Cao', verbose=0)

# Fit and predict clusters
clusters = kproto.fit_predict(kproto_prep_2.values, categorical=[1])

# Add cluster assignments to original data
kproto_prep_2['cluster'] = clusters

In [ ]:
kproto_prep_2

measure cluster size

In [ ]:
kproto_prep_2['cluster'].value_counts()

## Plot map

In [ ]:
plt.figure(figsize=(12,8))
plt.style.use('fast')

# Set the boundary of the map using longitude and latitude obtained from Google Maps
coordinates = (-74.2623, -73.6862, 40.4943, 40.9144)

map = mpimg.imread("New_York_City.jpg")
plt.imshow(map,extent=coordinates)

# Group by cluster labels instead of neighbourhood group
clusters = kproto_prep_2.groupby('cluster')

# Loop through each cluster and plot the listings in it
for name, group in clusters:
    plt.scatter(group['longitude'], group['latitude'], label=name, edgecolors='black')

plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Properties by Cluster')
plt.legend()

Place original lat & long back so the results can be plotted on the map

In [ ]:
kproto_prep_2_org_lat_long = kproto_prep_2.copy()

kproto_prep_2_org_lat_long['latitude'] = orig_lat
kproto_prep_2_org_lat_long['longitude'] = orig_long

kproto_prep_2_org_lat_long

In [ ]:
plt.figure(figsize=(12,8))
plt.style.use('fast')

# Set the boundary of the map using longitude and latitude obtained from Google Maps
coordinates = (-74.2623, -73.6862, 40.4943, 40.9144)

map = mpimg.imread("New_York_City.jpg")
plt.imshow(map,extent=coordinates)

# Group by cluster labels instead of neighbourhood group
clusters = kproto_prep_2_org_lat_long.groupby('cluster')

# Loop through each cluster and plot the listings in it
for name, group in clusters:
    plt.scatter(group['longitude'], group['latitude'], label=name, edgecolors='black')

plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Properties by Cluster')
plt.legend()

This looks interesting. Lets explore the data with summary statistics for each cluster group

In [ ]:
cluster_summary = kproto_prep_2_org_lat_long.groupby('cluster')['log_revenue_opportunity'].describe()
cluster_summary

In [ ]:
# Boxplot of 'log_price' for each cluster
plt.figure(figsize=(12, 8))
sns.boxplot(x='cluster', y='log_revenue_opportunity', data=kproto_prep_2_org_lat_long)
plt.title('Revenue Distribution per Cluster')
plt.show()

## ANOVA results

In [ ]:
anova_results = st.f_oneway(*(kproto_prep_2_org_lat_long['log_revenue_opportunity'][kproto_prep_2_org_lat_long['cluster'] == cluster] for cluster in kproto_prep_2_org_lat_long['cluster'].unique()))
anova_results

The above would suggest that the price is significantly different across the clusters.

In [ ]:
room_counts = kproto_prep_2_org_lat_long.groupby(['cluster', airbnb['room_type']]).size().unstack()
room_counts.plot(kind='bar', stacked=True)
plt.title('Counts of Room Types in Different Clusters')
plt.show()

## Add `neighbourhood_group` back to see which group is in which cluster

In [ ]:
# put neighbourhoods back
kproto_prep_2_org_lat_long['neighbourhood_group'] = airbnb['neighbourhood_group']

In [ ]:
kproto_prep_2_org_lat_long

In [ ]:
plt.figure(figsize=(10, 8))
sns.countplot(data=kproto_prep_2_org_lat_long, x='neighbourhood_group', hue='cluster')

plt.title('Distribution of Clusters within Each Neighbourhood Group')
plt.ylabel('Count')
plt.show()

# KMEANS clustering `revenue_opportunity`

Only left with numerical variables, thus run kmeans

In [ ]:
# create reduced dataframe
kmeans_run = airbnb[['log_revenue_opportunity']]
kmeans_run

Standardise (not really required with one variable, but still normalising for consistency)

In [ ]:
# standardise numeric variables
scaler = StandardScaler()
kmeans_run[['log_revenue_opportunity']] = scaler.fit_transform(kmeans_run[['log_revenue_opportunity']])

In [ ]:
kmeans_run

In [ ]:
kmeans_run.dtypes

In [ ]:
k_values = []
sil_scores = []
sq_distances = []

for i in range(2,13):
    # Initialize KMeans algorithm
    # 12 times per run to find the optimal centroids
    # random_state to ensure the same clusters every time we run this
    kmeans = KMeans(n_clusters=i, init='k-means++', n_init=12, random_state=0)

    # Fit and predict clusters
    clusters = kmeans.fit_predict(kmeans_run)

    # Compute silhouette score
    SScore = silhouette_score(kmeans_run, clusters, metric='euclidean')

    # Append to the lists
    k_values.append(i)
    sil_scores.append(SScore)
    sq_distances.append(kmeans.inertia_)  # Sum of squared distances to closest centroid

    print("Silhouette score for k (clusters) = " + str(i) + " is " + str(SScore))

# Plot silhouette scores
plt.figure(figsize=(10,5))
plt.subplot(1, 2, 1)
plt.plot(k_values, sil_scores, 'bx-')
plt.xlabel('k (number of clusters)')
plt.ylabel('Silhouette Score')
plt.title('Silhouette Score vs Number of Clusters')

# Plot sum of squared distances (for the elbow plot)
plt.subplot(1, 2, 2)
plt.plot(k_values, sq_distances, 'bx-')
plt.xlabel('k (number of clusters)')
plt.ylabel('Sum of Squared Distances')
plt.title('Elbow Plot (Sum of Squared Distances vs Number of Clusters)')

plt.tight_layout()
plt.show()

## now run kmeans with optimal clusters

### the elbow is at 5 but the silhouette score is higher with less clusters, meeting at 4

### continue analysis with 4 clusters

In [ ]:
# Initialize KMeans algorithm
# 12 times per run to find the optimal centroids
# random_state to ensure the same clusters every time we run this
kmeans_optimal = KMeans(n_clusters=4, init='k-means++', n_init=12, random_state=0)

# Fit and predict clusters
clusters_optimal = kmeans_optimal.fit_predict(kmeans_run)

In [ ]:
kmeans_run['cluster'] = clusters_optimal
kmeans_run

In [ ]:
kmeans_run['cluster'].value_counts()

Place original lat & long back so the results can be plotted on the map

In [ ]:
kmeans_run['latitude'] = orig_lat
kmeans_run['longitude'] = orig_long

In [ ]:
plt.figure(figsize=(12,8))
plt.style.use('fast')

# Set the boundary of the map using longitude and latitude obtained from Google Maps
coordinates = (-74.2623, -73.6862, 40.4943, 40.9144)

map = mpimg.imread("New_York_City.jpg")
plt.imshow(map,extent=coordinates)

# Group by cluster labels instead of neighbourhood group
clusters = kmeans_run.groupby('cluster')

# Loop through each cluster and plot the listings in it
for name, group in clusters:
    plt.scatter(group['longitude'], group['latitude'], label=name, edgecolors='black')

plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Properties by Cluster')
plt.legend()

Lets explore the data with summary statistics for each cluster group

In [ ]:
cluster_summary = kmeans_run.groupby('cluster')['log_revenue_opportunity'].describe()
print(cluster_summary)

In [ ]:
# Boxplot of 'log_price' for each cluster
plt.figure(figsize=(12, 8))
sns.boxplot(x='cluster', y='log_revenue_opportunity', data=kmeans_run)
plt.title('Revenue Distribution per Cluster')
plt.show()

## ANOVA results

In [ ]:
anova_results = st.f_oneway(*(kmeans_run['log_revenue_opportunity'][kmeans_run['cluster'] == cluster] for cluster in kmeans_run['cluster'].unique()))
anova_results

The above would suggest that the price is significantly different across the clusters.

# Add `neighbourhood_group` back to see which group is in which cluster

In [ ]:
# put neighbourhoods back
kmeans_run['neighbourhood_group'] = airbnb['neighbourhood_group']

In [ ]:
plt.figure(figsize=(10, 8))
sns.countplot(data=kmeans_run, x='neighbourhood_group', hue='cluster')

plt.title('Distribution of Clusters within Each Neighbourhood Group')
plt.ylabel('Count')
plt.show()

# DBSCAN clustering

In [ ]:
# keeping less than we're dropping so just picking those features
cluster_data = airbnb[['price', 'log_price', 'neighbourhood_group', 'latitude', 'longitude', 'room_type', 'revenue_opportunity', 'log_revenue_opportunity', 'availability_365']]
cluster_data

In [ ]:
# Create a copy of the data
cluster_data_prepared = cluster_data.copy()

# Encode categorical variables
le = LabelEncoder()
cluster_data_prepared['room_type_xform'] = le.fit_transform(cluster_data['room_type'])

cluster_data_prepared

Plot a k-distance graph to set eps as the point of maximum curvture

In [ ]:
neigh = NearestNeighbors(n_neighbors=2)
nbrs = neigh.fit(cluster_data_prepared[['log_price','room_type_xform']])
distances, indices = nbrs.kneighbors(cluster_data_prepared[['log_price','room_type_xform']])

In [ ]:
# Plotting K-distance Graph
distances = np.sort(distances, axis=0)
distances = distances[:,1]
plt.figure(figsize=(20,10))
plt.plot(distances)
plt.title('K-distance Graph',fontsize=20)
plt.xlabel('Data Points sorted by distance',fontsize=14)
plt.ylabel('Epsilon',fontsize=14)
plt.show()

This doesn't help much other than confirming the value of eps should be very small. It looks like 0.01 or possibly even smaller.

After experimentation an eps of 0.1 gave good results

In [ ]:
dbscan=DBSCAN(eps=0.1,min_samples=9)
dbscan.fit(cluster_data_prepared[['log_price','room_type_xform']])

In [ ]:
# Get the labels of the DBSCAN clustering
labels = dbscan.labels_

# Compute the silhouette score
silhouette_avg = silhouette_score(cluster_data_prepared[['log_price','room_type_xform']], labels)

print("The average silhouette_score for DBSCAN is :", silhouette_avg)


In [ ]:
cluster_data_prepared['DBSCAN_labels']=dbscan.labels_
plt.figure(figsize=(10,10))
plt.scatter(cluster_data_prepared['log_price'],cluster_data_prepared['room_type_xform'],c=cluster_data_prepared['DBSCAN_labels'])
plt.title('DBSCAN Clustering',fontsize=20)
plt.xlabel('Feature 1',fontsize=14)
plt.ylabel('Feature 2',fontsize=14)
plt.show()

In [ ]:
cluster_data_prepared

In [ ]:
plt.figure(figsize=(12,8))
plt.style.use('fast')
# Set the boundary of the map using longitude and latitude obtained from Google Maps
coordinates = (-74.2623, -73.6862, 40.4943, 40.9144)
map = mpimg.imread("New_York_City.jpg")
plt.imshow(map,extent=coordinates)
groups = cluster_data_prepared.groupby('DBSCAN_labels')
for name,group in groups :
     plt.scatter(group['longitude'],group['latitude'], label=name, edgecolors='black')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Properties by cluster')
plt.legend()

Export to a file to see how the clusters are made up

In [ ]:
cluster_data_prepared.to_csv('clusters.csv', index=False)

Create a separate dataframe for each cluster and plot them separately

In [ ]:
Cluster_1 = cluster_data_prepared[cluster_data_prepared['DBSCAN_labels'] == -1]
Cluster0 = cluster_data_prepared[cluster_data_prepared['DBSCAN_labels'] == 0]
Cluster1 = cluster_data_prepared[cluster_data_prepared['DBSCAN_labels'] == 1]
Cluster2 = cluster_data_prepared[cluster_data_prepared['DBSCAN_labels'] == 2]
Cluster3 = cluster_data_prepared[cluster_data_prepared['DBSCAN_labels'] == 3]
Cluster4 = cluster_data_prepared[cluster_data_prepared['DBSCAN_labels'] == 4]
Cluster5 = cluster_data_prepared[cluster_data_prepared['DBSCAN_labels'] == 5]

Check room types per cluster

In [ ]:
plt.figure(figsize=(10, 8))
sns.countplot(data=cluster_data_prepared, x='DBSCAN_labels', hue='room_type')

plt.title('Distribution of Clusters within Each Neighbourhood Group')
plt.ylabel('Count')
plt.show()

In [ ]:
print("Cluster -1 room types:", pd.unique(Cluster_1['room_type']))
print("Cluster 0 room types:", pd.unique(Cluster0['room_type']))
print("Cluster 1 room types:", pd.unique(Cluster1['room_type']))
print("Cluster 2 room types:", pd.unique(Cluster2['room_type']))
print("Cluster 3 room types:", pd.unique(Cluster3['room_type']))
print("Cluster 4 room types:", pd.unique(Cluster4['room_type']))
print("Cluster 5 room types:", pd.unique(Cluster5['room_type']))

We can see that other than the outliers, each cluster has just one room type

Boxplot of price for each cluster

In [ ]:
plt.figure(figsize=(12, 8))
sns.boxplot(x='DBSCAN_labels', y='price', data=cluster_data_prepared)
plt.title('Price Distribution per Cluster')
plt.axis(ymin=0, ymax=5000)
plt.show()

We can also see clustering based upon price.

So the clusters are based upon room type and price:
Cluster 0 = Private room priced 20 - 1,200
Cluster 1 = Entire home/apt priced 34 - 1,800
Cluster 2 = Shared room priced 19 - 165
Cluster 3 = Entire home/apt priced 1,999 - 3,518
Cluster 4 = Shared room priced 195 - 235
Cluster 5 = Entire home/apt priced 4,000 - 4,500

Now we plot the clusters on the map

In [ ]:
# Cluster 0
plt.figure(figsize=(12,8))
plt.style.use('fast')
# Set the boundary of the map using longitude and latitude obtained from Google Maps
coordinates = (-74.2623, -73.6862, 40.4943, 40.9144)
map = mpimg.imread("New_York_City.jpg")
plt.imshow(map,extent=coordinates)

plt.scatter(Cluster0['longitude'],Cluster0['latitude'], edgecolors='black')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Properties in cluster 0: Private room priced 20 - 1,200')
plt.legend()

In [ ]:
# Cluster 1
plt.figure(figsize=(12,8))
plt.style.use('fast')
# Set the boundary of the map using longitude and latitude obtained from Google Maps
coordinates = (-74.2623, -73.6862, 40.4943, 40.9144)
map = mpimg.imread("New_York_City.jpg")
plt.imshow(map,extent=coordinates)

plt.scatter(Cluster1['longitude'],Cluster1['latitude'], edgecolors='black')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Properties in cluster 1: Entire home/apt priced 34 - 1,800')
plt.legend()

In [ ]:
# Clusters 2 and 4
plt.figure(figsize=(12,8))
plt.style.use('fast')
# Set the boundary of the map using longitude and latitude obtained from Google Maps
coordinates = (-74.2623, -73.6862, 40.4943, 40.9144)
map = mpimg.imread("New_York_City.jpg")
plt.imshow(map,extent=coordinates)
cluster_2_and_4 = cluster_data_prepared[(cluster_data_prepared['DBSCAN_labels'] == 2) | (cluster_data_prepared['DBSCAN_labels'] == 4)]
groups = cluster_2_and_4.groupby('DBSCAN_labels')
for name,group in groups :
     plt.scatter(group['longitude'],group['latitude'], label=name, edgecolors='black')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Properties in clusters 2 (Shared room priced 19 - 165) and 4 (Shared room priced 195 - 235)')
plt.legend()

In [ ]:
# Clusters 3 and 5
plt.figure(figsize=(12,8))
plt.style.use('fast')
# Set the boundary of the map using longitude and latitude obtained from Google Maps
coordinates = (-74.2623, -73.6862, 40.4943, 40.9144)
map = mpimg.imread("New_York_City.jpg")
plt.imshow(map,extent=coordinates)
cluster_3_and_5 = cluster_data_prepared[(cluster_data_prepared['DBSCAN_labels'] == 3) | (cluster_data_prepared['DBSCAN_labels'] == 5)]
groups = cluster_3_and_5.groupby('DBSCAN_labels')
for name,group in groups :
     plt.scatter(group['longitude'],group['latitude'], label=name, edgecolors='black')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Properties in clusters 3 (Entire home/apt priced 1,999 - 3,518) and 5 (Entire home/apt priced 4,000 - 4,500)')
plt.legend()

## DBSCAN using revenue opportunity, room type and neighbourhood group

In [ ]:
le = LabelEncoder()
cluster_data_prepared['neighbourhood_group_xform'] = le.fit_transform(cluster_data['neighbourhood_group'])

In [ ]:
dbscan=DBSCAN(eps=0.9,min_samples=9)
dbscan.fit(cluster_data_prepared[['log_revenue_opportunity','room_type_xform','neighbourhood_group_xform']])

In [ ]:
# Get the labels of the DBSCAN clustering
labels = dbscan.labels_

# Compute the silhouette score
silhouette_avg = silhouette_score(cluster_data_prepared[['log_revenue_opportunity', 'room_type_xform', 'neighbourhood_group_xform']], labels)

print("The average silhouette_score for DBSCAN is :", silhouette_avg)


In [ ]:
cluster_data_prepared['DBSCAN_labels']=dbscan.labels_
plt.figure(figsize=(10,10))
plt.scatter(cluster_data_prepared['log_revenue_opportunity'],cluster_data_prepared['room_type_xform'],cluster_data_prepared['neighbourhood_group_xform'],c=cluster_data_prepared['DBSCAN_labels'])
plt.title('DBSCAN Clustering',fontsize=20)
plt.xlabel('Feature 1',fontsize=14)
plt.ylabel('Feature 2',fontsize=14)
plt.show()

In [ ]:
cluster_data_prepared

In [ ]:
plt.figure(figsize=(12,8))
plt.style.use('fast')
# Set the boundary of the map using longitude and latitude obtained from Google Maps
coordinates = (-74.2623, -73.6862, 40.4943, 40.9144)
map = mpimg.imread("New_York_City.jpg")
plt.imshow(map,extent=coordinates)
groups = cluster_data_prepared.groupby('DBSCAN_labels')
for name,group in groups :
     plt.scatter(group['longitude'],group['latitude'], label=name, edgecolors='black')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Properties by cluster')
plt.legend()

Plot only Queens, having already identified it as having the highest potential revenue opportunity.

In [ ]:
plt.figure(figsize=(12,8))
plt.style.use('fast')
# Set the boundary of the map using longitude and latitude obtained from Google Maps
coordinates = (-74.2623, -73.6862, 40.4943, 40.9144)
map = mpimg.imread("New_York_City.jpg")
plt.imshow(map,extent=coordinates)
Queens = cluster_data_prepared[(cluster_data_prepared['DBSCAN_labels'] == 5) | (cluster_data_prepared['DBSCAN_labels'] == 8) | (cluster_data_prepared['DBSCAN_labels'] == 11)]
groups = Queens.groupby('DBSCAN_labels')
for name,group in groups :
     plt.scatter(group['longitude'],group['latitude'], label=name, edgecolors='black')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Properties by cluster')
plt.legend()

In [ ]:
plt.figure(figsize=(12, 8))
sns.boxplot(x='DBSCAN_labels', y='price', data=cluster_data_prepared)
plt.title('Price Distribution per Cluster')
plt.axis(ymin=0, ymax=500)
plt.show()

In [ ]:
plt.figure(figsize=(12, 8))
sns.boxplot(x='DBSCAN_labels', y='revenue_opportunity', data=cluster_data_prepared)
plt.title('Revenue opportunity Distribution per Cluster')
plt.axis(ymin=0, ymax=15000)
plt.show()

In [ ]:
plt.figure(figsize=(12, 8))
sns.boxplot(x='DBSCAN_labels', y='availability_365', data=cluster_data_prepared)
plt.title('Availability Distribution per Cluster')
plt.show()

In [ ]:
plt.figure(figsize=(10, 8))
sns.countplot(data=cluster_data_prepared, x='DBSCAN_labels', hue='neighbourhood_group')

plt.title('Distribution of Clusters within Each Neighbourhood Group')
plt.ylabel('Count')
plt.show()

In [ ]:
Cluster_1 = cluster_data_prepared[cluster_data_prepared['DBSCAN_labels'] == -1]
Cluster0 = cluster_data_prepared[cluster_data_prepared['DBSCAN_labels'] == 0]
Cluster1 = cluster_data_prepared[cluster_data_prepared['DBSCAN_labels'] == 1]
Cluster2 = cluster_data_prepared[cluster_data_prepared['DBSCAN_labels'] == 2]
Cluster3 = cluster_data_prepared[cluster_data_prepared['DBSCAN_labels'] == 3]
Cluster4 = cluster_data_prepared[cluster_data_prepared['DBSCAN_labels'] == 4]
Cluster5 = cluster_data_prepared[cluster_data_prepared['DBSCAN_labels'] == 5]
Cluster6 = cluster_data_prepared[cluster_data_prepared['DBSCAN_labels'] == 6]
Cluster7 = cluster_data_prepared[cluster_data_prepared['DBSCAN_labels'] == 7]
Cluster8 = cluster_data_prepared[cluster_data_prepared['DBSCAN_labels'] == 8]
Cluster9 = cluster_data_prepared[cluster_data_prepared['DBSCAN_labels'] == 9]
Cluster10 = cluster_data_prepared[cluster_data_prepared['DBSCAN_labels'] == 10]
Cluster11 = cluster_data_prepared[cluster_data_prepared['DBSCAN_labels'] == 11]
Cluster12 = cluster_data_prepared[cluster_data_prepared['DBSCAN_labels'] == 12]
Cluster13 = cluster_data_prepared[cluster_data_prepared['DBSCAN_labels'] == 13]

In [ ]:
print("Cluster -1 room types:", pd.unique(Cluster_1['room_type']), "neighbourhood groups:", pd.unique(Cluster_1['neighbourhood_group']))
print("Cluster 0 room types:", pd.unique(Cluster0['room_type']), "neighbourhood groups:", pd.unique(Cluster0['neighbourhood_group']))
print("Cluster 1 room types:", pd.unique(Cluster1['room_type']), "neighbourhood groups:", pd.unique(Cluster1['neighbourhood_group']))
print("Cluster 2 room types:", pd.unique(Cluster2['room_type']), "neighbourhood groups:", pd.unique(Cluster2['neighbourhood_group']))
print("Cluster 3 room types:", pd.unique(Cluster3['room_type']), "neighbourhood groups:", pd.unique(Cluster3['neighbourhood_group']))
print("Cluster 4 room types:", pd.unique(Cluster4['room_type']), "neighbourhood groups:", pd.unique(Cluster4['neighbourhood_group']))
print("Cluster 5 room types:", pd.unique(Cluster5['room_type']), "neighbourhood groups:", pd.unique(Cluster5['neighbourhood_group']))
print("Cluster 6 room types:", pd.unique(Cluster6['room_type']), "neighbourhood groups:", pd.unique(Cluster6['neighbourhood_group']))
print("Cluster 7 room types:", pd.unique(Cluster7['room_type']), "neighbourhood groups:", pd.unique(Cluster7['neighbourhood_group']))
print("Cluster 8 room types:", pd.unique(Cluster8['room_type']), "neighbourhood groups:", pd.unique(Cluster8['neighbourhood_group']))
print("Cluster 9 room types:", pd.unique(Cluster9['room_type']), "neighbourhood groups:", pd.unique(Cluster9['neighbourhood_group']))
print("Cluster 10 room types:", pd.unique(Cluster10['room_type']), "neighbourhood groups:", pd.unique(Cluster10['neighbourhood_group']))
print("Cluster 11 room types:", pd.unique(Cluster11['room_type']), "neighbourhood groups:", pd.unique(Cluster11['neighbourhood_group']))
print("Cluster 12 room types:", pd.unique(Cluster12['room_type']), "neighbourhood groups:", pd.unique(Cluster12['neighbourhood_group']))
print("Cluster 13 room types:", pd.unique(Cluster13['room_type']), "neighbourhood groups:", pd.unique(Cluster13['neighbourhood_group']))

In [ ]:
cluster_summary = cluster_data_prepared.groupby('DBSCAN_labels')['revenue_opportunity'].describe()
print(cluster_summary)

In order of number of rooms available:

Cluster 1 (Entire home/apt in Manhattan) has the highest median revenue opportunity with a relatively low availability, so increasing the availability would significantly increase revenue here.

Cluster 3 (Entire home/apt in Brooklyn) has the lowest median availability, moderately high price and high revenue opportunity. Increasing availability and number of customers in this segment would help.

Cluster 0 (Private room in Brooklyn) has a low price, low availability and low revenue opportunity even though there is a high count of this property type. This does not seem like a profitable segment to push right now.

Cluster 2 (Private room in Mahattan) has moderate price, availability and revenue opportunity.

Cluster 5 (Private room in Queens) has good availability but low price and low revenue opportunity.

Cluster 8 (Entire home/apt in Queens) good availability and high price and high revenue opportunity.

Cluster 7 (Private room on Bronx) has low price, reasonable availability and low revenue opportunity.

Cluster 4 (Shared room in Manhattan) has low price, reasonable availability and low revenue opportunity.

Cluster 9 (Entire home/apt in Bronx) has moderate price, availability and revenue opportunity.

Cluster 12 (Shared room in Brooklyn) has very low price, high availability but low revenue opportunity.

Cluster 6 (Private room in Staten Island) has low price, high availability and low revenue opportunity.

Cluster 11 (Shared room in Queens) has very low price, high availability but low revenue opportunity.

Cluster 10 (Entire home/apt in Staten Island) has moderate price and availability and quite high revenue opportunity.

Cluster 13 (Shared room in Bronx) has very low price, low availability and very low revenue opportunity.

Cluster -1 contains 43 outliers across all neighbourhood regions effectively "dropped"



We have already established that Queens is a good place to target in order to increase the number of listed rooms per resident. We also established earlier that Queens has disproportionalely fewer Entire home/appt listings than Manhattan or Brooklyn. Here we see that those Entire home/appt listings that we do have in Queens have a lower price than Manhattan and Brooklyn, but availability similar to Manhatan and higher than Brooklyn, making their revenue opportunity similar to Brooklyn, although still lower than Manhattan. So based upon all of those factors, a focus for Airbnb could be increasing the number of Entire home/appt listings in Queens.


# KMEANS clustering `revenue_opportunity` in Queens

Let's see what Queens looks like clustered on revenue_oportunity.

It's a single numerical variable so we can use k-means.

In [ ]:
# create reduced dataframe
kmeans_run = Queens[['log_revenue_opportunity']]
kmeans_run

Standardise (not really required with one variable, but still normalising for consistency)

In [ ]:
# standardise numeric variables
scaler = StandardScaler()
kmeans_run[['log_revenue_opportunity']] = scaler.fit_transform(kmeans_run[['log_revenue_opportunity']])

In [ ]:
kmeans_run

In [ ]:
kmeans_run.dtypes

In [ ]:
k_values = []
sil_scores = []
sq_distances = []

for i in range(2,13):
    # Initialize KMeans algorithm
    # 12 times per run to find the optimal centroids
    # random_state to ensure the same clusters every time we run this
    kmeans = KMeans(n_clusters=i, init='k-means++', n_init=12, random_state=0)

    # Fit and predict clusters
    clusters = kmeans.fit_predict(kmeans_run)

    # Compute silhouette score
    SScore = silhouette_score(kmeans_run, clusters, metric='euclidean')

    # Append to the lists
    k_values.append(i)
    sil_scores.append(SScore)
    sq_distances.append(kmeans.inertia_)  # Sum of squared distances to closest centroid

    print("Silhouette score for k (clusters) = " + str(i) + " is " + str(SScore))

# Plot silhouette scores
plt.figure(figsize=(10,5))
plt.subplot(1, 2, 1)
plt.plot(k_values, sil_scores, 'bx-')
plt.xlabel('k (number of clusters)')
plt.ylabel('Silhouette Score')
plt.title('Silhouette Score vs Number of Clusters')

# Plot sum of squared distances (for the elbow plot)
plt.subplot(1, 2, 2)
plt.plot(k_values, sq_distances, 'bx-')
plt.xlabel('k (number of clusters)')
plt.ylabel('Sum of Squared Distances')
plt.title('Elbow Plot (Sum of Squared Distances vs Number of Clusters)')

plt.tight_layout()
plt.show()

## now run kmeans with optimal clusters

### the elbow is at 5 but the silhouette score is higher with less clusters, meeting at 4

### continue analysis with 4 clusters

In [ ]:
# Initialize KMeans algorithm
# 12 times per run to find the optimal centroids
# random_state to ensure the same clusters every time we run this
kmeans_optimal = KMeans(n_clusters=4, init='k-means++', n_init=12, random_state=0)

# Fit and predict clusters
clusters_optimal = kmeans_optimal.fit_predict(kmeans_run)

In [ ]:
kmeans_run['cluster'] = clusters_optimal
kmeans_run

In [ ]:
kmeans_run['cluster'].value_counts()

Place original lat & long back so the results can be plotted on the map

In [ ]:
kmeans_run['latitude'] = Queens[['latitude']]
kmeans_run['longitude'] = Queens[['longitude']]

In [ ]:
plt.figure(figsize=(12,8))
plt.style.use('fast')

# Set the boundary of the map using longitude and latitude obtained from Google Maps
coordinates = (-74.2623, -73.6862, 40.4943, 40.9144)

map = mpimg.imread("New_York_City.jpg")
plt.imshow(map,extent=coordinates)

# Group by cluster labels instead of neighbourhood group
clusters = kmeans_run.groupby('cluster')

# Loop through each cluster and plot the listings in it
for name, group in clusters:
    plt.scatter(group['longitude'], group['latitude'], label=name, edgecolors='black')

plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Properties by Cluster')
plt.legend()

Lets explore the data with summary statistics for each cluster group

In [ ]:
cluster_summary = kmeans_run.groupby('cluster')['log_revenue_opportunity'].describe()
print(cluster_summary)

In [ ]:
latitude_mean = kmeans_run.groupby('cluster')['latitude'].mean()
print(latitude_mean)

In [ ]:
longitude_mean = kmeans_run.groupby('cluster')['longitude'].mean()
print(longitude_mean)

In [ ]:
log_rev_opp__mean = kmeans_run.groupby('cluster')['log_revenue_opportunity'].mean()
print(log_rev_opp__mean)

## table rev opp log

In [ ]:
# Convert series to df
kmeans_queens = log_rev_opp__mean.reset_index()

# Merge the mean latitude and longitude with the original DataFrame
kmeans_queens = kmeans_queens.merge(latitude_mean, on='cluster', how='left')
kmeans_queens = kmeans_queens.merge(longitude_mean, on='cluster', how='left')

# Rename columns
kmeans_queens.columns = ['cluster', 'Mean Log Rev Opp', 'Mean Latitude', 'Mean Longitude']
kmeans_queens = kmeans_queens.sort_values(by=['Mean Log Rev Opp'])

kmeans_queens

In [ ]:
styled_df = kmeans_queens.style.background_gradient(
    cmap='Blues', subset=['Mean Log Rev Opp', 'Mean Latitude', 'Mean Longitude']
).hide_index().set_table_styles(
    [
        {
            'selector': 'th',
            'props': [
                ('padding', '1px'),
                ('text-align', 'left')
            ]
        },
        {
            'selector': 'td',
            'props': [
                ('padding', '1px'),
                ('text-align', 'center')
            ]
        }
    ]
).set_properties(**{'width': '70px'})

styled_df

In [ ]:
# Boxplot of 'log_price' for each cluster
plt.figure(figsize=(12, 8))
sns.boxplot(x='cluster', y='log_revenue_opportunity', data=kmeans_run)
plt.title('Revenue Distribution per Cluster')
plt.show()

## ANOVA results

In [ ]:
anova_results = st.f_oneway(*(kmeans_run['log_revenue_opportunity'][kmeans_run['cluster'] == cluster] for cluster in kmeans_run['cluster'].unique()))
anova_results

The above would suggest that the price is significantly different across the clusters.

## LINEAR REGRESSION

Let's see if we can create a model using linear regression to predict where within Queens (the neighbourhood group identified as having the highest revenue potrential) Airbnb could see the most revenue when adding additional properties.

## Longitude and log revenue oportunity

First we'll check longitude against revenue opportunity using the Queens dataframe created from the three clusters in the last DBSCAN.

In [ ]:
msk=np.random.rand(len(Queens))<0.8
train=Queens[msk]
test=Queens[~msk]

In [ ]:
# Train data distribution
plt.scatter(train.longitude,train.log_revenue_opportunity, color='blue')
plt.xlabel("Longitude")
plt.ylabel("Log revenue opportunity")
plt.show()

## Using sklearn package for data modelling

In [ ]:
regr=linear_model.LinearRegression()
train_x=np.asanyarray(train[['longitude']])
train_y=np.asanyarray(train[['log_revenue_opportunity']])

regr.fit(train_x, train_y)
# The coefficients
print('Coefficients:', regr.coef_)
print('Intercept:', regr.intercept_)

In [ ]:
# Plot outputs
plt.scatter(train.longitude,train.log_revenue_opportunity,color='blue')
plt.plot(train_x,regr.coef_[0][0]*train_x + regr.intercept_[0],'-r')
plt.xlabel("Longitude")
plt.ylabel("Log revenue opportunity")

## Model evaluation

In [ ]:
test_x=np.asanyarray(test[['longitude']])
test_y=np.asanyarray(test[['log_revenue_opportunity']])
test_y_ = regr.predict(test_y)

In [ ]:
print("Mean absolute error: %.2f" % np.mean(np.absolute(test_y_-test_y)))
print("Residual sum of squares (MSE): %.2f" % np.mean((test_y_-test_y)**2))
print("R2-score: %.2f" % r2_score(test_y_,test_y))

In [ ]:
print(r2_score(train_x,train_y))

In [ ]:
pred_y = regr.predict(test_x)

In [ ]:
df = pd.DataFrame({'Actual': test_y.flatten(), 'Predicted': pred_y.flatten()})
df1 = df.head(50)
df1.plot(kind='bar',figsize=(16,10))
plt.grid(which='major', linestyle='-', linewidth='0.5', color='green')
plt.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
plt.show()

## Latitude and log revenue oportunity

Next we'll check latitude against revenue opportunity using the Queens dataframe created from the three clusters in the last DBSCAN.

In [ ]:
msk=np.random.rand(len(Queens))<0.8
train=Queens[msk]
test=Queens[~msk]

In [ ]:
# Train data distribution
plt.scatter(train.latitude,train.log_revenue_opportunity, color='blue')
plt.xlabel("Latitude")
plt.ylabel("Log revenue opportunity")
plt.show()

## Using sklearn package for data modelling

In [ ]:
regr=linear_model.LinearRegression()
train_x=np.asanyarray(train[['latitude']])
train_y=np.asanyarray(train[['log_revenue_opportunity']])

regr.fit(train_x, train_y)
# The coefficients
print('Coefficients:', regr.coef_)
print('Intercept:', regr.intercept_)

In [ ]:
# Plot outputs
plt.scatter(train.latitude,train.log_revenue_opportunity,color='blue')
plt.plot(train_x,regr.coef_[0][0]*train_x + regr.intercept_[0],'-r')
plt.xlabel("Latitude")
plt.ylabel("Log revenue opportunity")

## Model evaluation

In [ ]:
test_x=np.asanyarray(test[['latitude']])
test_y=np.asanyarray(test[['log_revenue_opportunity']])
test_y_ = regr.predict(test_y)

In [ ]:
print("Mean absolute error: %.2f" % np.mean(np.absolute(test_y_-test_y)))
print("Residual sum of squares (MSE): %.2f" % np.mean((test_y_-test_y)**2))
print("R2-score: %.2f" % r2_score(test_y_,test_y))

In [ ]:
print(r2_score(train_x,train_y))

In [ ]:
pred_y = regr.predict(test_x)

In [ ]:
df = pd.DataFrame({'Actual': test_y.flatten(), 'Predicted': pred_y.flatten()})
df1 = df.head(50)
df1.plot(kind='bar',figsize=(16,10))
plt.grid(which='major', linestyle='-', linewidth='0.5', color='green')
plt.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
plt.show()

We can see that neither latitude or longitude gave a reliable preduction for revenue.

There was, however, a weak correlation between both longitude and latitude and revenue_opportunity.

Revenue opportunity trended slightly higher with a higher longitude and with a lower latitude, suggesting that revenue opportunity is slightly higher further West and South in Queens. As already stated, the correlation was weak, however, and could not be used to predict actual revenues.